In [1]:
%pip install -q supervision
%pip install -q git+https://github.com/THU-MIG/yolov10.git


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
%pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 2.1 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


# Google collab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

# Генерация датасета

In [11]:
!mkdir -p dataset
!unzip /content/drive/MyDrive/dataset.zip -d dataset

�������� ��� 䠩� -p 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: -p.
�������� ��� 䠩� dataset 㦥 �������.
�訡�� �� �६� ��ࠡ�⪨: dataset.
"unzip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [ ]:
import glob
import os
import random
from tqdm import tqdm
import shutil
import cv2
from ultralytics import YOLOv10

In [2]:
path_to_dataset = 'dataset'

In [3]:
#!rm -rf train/ val/
for path in ['train', 'val']:
    os.makedirs(os.path.join('images', path), exist_ok=True)
    os.makedirs(os.path.join('labels', path), exist_ok=True)

In [3]:
val_prob = 0.1
for jpg in tqdm(glob.glob(os.path.join(path_to_dataset, '*.jpg'))):
    folder = 'train' if random.random() > val_prob else 'val'
    shutil.copyfile(jpg, os.path.join('images', folder, os.path.basename(jpg)))
for jpg in tqdm(glob.glob(os.path.join(path_to_dataset, '*.txt'))):
    folder = 'train' if random.random() > val_prob else 'val'
    shutil.copyfile(jpg, os.path.join('labels', folder, os.path.basename(jpg)))

100%|██████████| 1163/1163 [00:01<00:00, 1040.23it/s]


In [3]:
!cat ./svarka.yaml

"cat" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


# Обучение

In [6]:
!pip install ultralytics


In [16]:
from ultralytics import YOLOv10

model = YOLOv10('./yolov10x.pt')

In [15]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

147697

In [18]:
results = model.train(data='svarka.yaml', epochs=50, imgsz=1080, batch=4)

New https://pypi.org/project/ultralytics/8.2.32 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov10x.pt, data=svarka.yaml, epochs=500, time=None, patience=100, batch=4, imgsz=1080, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=yolov10x, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=yolov10x.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

train: Scanning /home/jupyter/work/resources/atomic-hack/ml/labels/train.cache... 926 images, 218 backgrounds, 0 corrupt: 100%|██████████| 1030/1030 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /home/jupyter/work/resources/atomic-hack/ml/labels/val.cache... 12 images, 124 backgrounds, 0 corrupt: 100%|██████████| 132/132 [00:00<?, ?it/s]


Plotting labels to runs/detect/yolov10x/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 185 weight(decay=0.0), 198 weight(decay=0.0005), 197 bias(decay=0.0)


AssertionError: yolov10x.pt training to 500 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolov10x.pt'

In [3]:
results

NameError: name 'results' is not defined

In [23]:
import matplotlib.pyplot as plt
from matplotlib import patches

image = cv2.imread('images/train/1 (1).jpg')
results = model.predict(source='images/train/1 (1).jpg', conf=0.02)
fig, ax = plt.subplots(figsize=(15,10))

ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

for box in results[0].boxes:
    x, y, w, h = box.xywh[0]
    class_name = box.id

    rect = patches.Rectangle((x, y), w, h, linewidth=1,
    edgecolor='r', facecolor='none')

    ax.add_patch(rect)

    ax.text(x, y, class_name, fontsize=12, color='r')

plt.show()


image 1/1 c:\Users\witcheredd\Downloads\images\train\1 (1).jpg: 384x640 3 adjs, 3 geos, 88.1ms
Speed: 5.0ms preprocess, 88.1ms inference, 18.0ms postprocess per image at shape (1, 3, 384, 640)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.